In [17]:
import numpy as np
import pandas as pd
import os 

In [18]:
aas_residualmz={"G":57.02147,"A":71.03712,"S":87.03203,"P":97.05277,"V":99.06842,"T":101.04768,"C":103.00919,"I":113.08407,"L":113.08407,"N":114.04293
,"D":115.02695,"Q":128.05858,"K":128.09497,"E":129.0426,"M":131.04049,"H":137.05891,"F":147.06842,"R":156.10112,"Y":163.06333,"W":186.07932}
aas=['A' ,'C', 'D' ,'E', 'F', 'G', 'H', 'I' ,'K', 'L', 'M', 'N' ,'P' ,'Q' ,'R' ,'S', 'T', 'V', 'W' ,'Y']

In [19]:
path="C:\\Workplace\\C#\\Test\\immonium_mass_neh\\immonium_mass\\bin\\x64\\Debug\\"

data=pd.DataFrame()
for f in ["Liver_l0.csv"]:    
    data=pd.concat([data,pd.read_csv(path+f)])
    
    
data["ion_count"]=[ len(str(x).split('|')) for x in data[["values"]].values]
print(data.shape)
data=data[data.ion_count>=2]
data=data.reset_index(drop=True)
print(data.shape)

(26187, 9)
(15339, 9)


In [20]:
all_mz=pd.DataFrame(np.unique([ int(float(j)*100)/100 for i in [x for x in [x.split("|") for x in data["values"]] ] for j in i], return_counts=True)).transpose()
all_mz.columns=["mz","freq"]
all_mz=all_mz[all_mz.freq>100]

In [5]:
dict_res={x:{"count":0,"total":0,"aas":[],"peps":[],"all_mzs":[]} for x in all_mz.mz}
count=0
for i in range(data.shape[0]): 
    temp=np.unique([ int(float(x)*100)/100 for x in data["values"][i].split("|")])
    temp=[x for x in temp if x in list(all_mz.mz)]
    if len(temp)==0:
        continue
        
    for mz in all_mz[all_mz.mz>= min(temp)].mz:
        dict_res[mz]["total"]+=1        
    
    for mz in temp:
        dict_res[mz]["count"]+=1
        dict_res[mz]["aas"]+=list(np.unique(list(data["peptide"][i])))
        dict_res[mz]["peps"]+=[data["peptide"][i]]
        dict_res[mz]["all_mzs"]+=list(np.unique(temp))
    

In [6]:
# np.unique(dict_res[list(dict_res.keys())[0]]["all_mzs"],return_counts=True)

In [7]:
# selected_res=[]
# for k in dict_res.keys():
# #     print(k,dict_res[k]["count"],dict_res[k]["total"],"\t===>\t",100*dict_res[k]["count"]/dict_res[k]["total"])
#     selected_res.append([k,dict_res[k]["count"],dict_res[k]["total"],100*dict_res[k]["count"]/dict_res[k]["total"]])

In [8]:
# selected_res=pd.DataFrame(selected_res)
# selected_res.columns=["mz","freq","Total","perc"]
# selected_res=selected_res.sort_values(by="perc",ascending=False).reset_index(drop=True)
# selected_res.to_csv("possible_mz.csv",index=False)
# selected_res.head(50)

In [12]:
dict_res_mzs=dict_res

In [9]:
selected_res=[]
for k in dict_res.keys():
    
    temp=(pd.DataFrame(np.unique(dict_res[k]["all_mzs"] ,return_counts=True)).transpose())
    temp.columns=["mz","perc"]
    temp.perc=100*temp.perc/dict_res[k]["count"]
    temp=temp.sort_values(by="perc",ascending=False)
    temp=(temp[temp.perc>30].reset_index(drop=True))
    if (temp.shape[0]>0):
        print("mz:",k,"count:",f'{dict_res[k]["count"]}({int(1000*dict_res[k]["count"]/dict_res[k]["total"])/10})',"Range available:",dict_res[k]["total"],"\n")
        print(temp[temp.mz!=k].head(12))
        print("------------------------------------------\n")
  

mz: 65.39 count: 185(100.0) Range available: 185 

        mz       perc
1    86.09  99.459459
2    70.06  98.378378
3    72.08  97.297297
4   129.10  95.675676
5   120.08  94.594595
6    84.08  93.513514
7   110.07  88.108108
8    84.04  76.216216
9    87.10  69.189189
10   74.06  62.162162
11  112.65  61.081081
12  101.07  60.540541
------------------------------------------

mz: 69.07 count: 383(76.1) Range available: 503 

        mz        perc
0    86.09  100.000000
2    70.06   97.389034
3    72.08   96.344648
4   129.10   96.083551
5    84.08   94.778068
6   120.08   91.644909
7    87.10   91.644909
8   110.07   86.422977
9    84.04   78.590078
10  101.07   66.057441
11   74.06   63.185379
12  102.05   61.879896
------------------------------------------

mz: 70.02 count: 135(23.3) Range available: 577 

        mz        perc
1    86.09  100.000000
2   129.10   97.777778
3    70.06   97.777778
4    72.08   97.777778
5    84.08   97.777778
6   120.08   95.555556
7   110.07   89

mz: 84.08 count: 4008(93.0) Range available: 4309 

        mz       perc
1    86.09  98.827345
2   129.10  98.602794
3   110.07  90.069860
4   120.08  85.828343
5    84.04  81.686627
6   101.07  71.282435
7   102.05  62.400200
8    87.10  55.838323
9    99.79  51.546906
10  104.05  50.673653
11   89.01  43.812375
12  112.65  43.213573
------------------------------------------

mz: 85.04 count: 135(3.1) Range available: 4318 

        mz        perc
1    86.09  100.000000
2   129.10   94.814815
3   120.08   94.074074
4   110.07   94.074074
5    84.08   90.370370
6    84.04   77.037037
7   101.07   72.592593
8   102.05   71.851852
9    87.10   69.629630
10   87.05   57.037037
11   99.79   57.037037
12  104.05   56.296296
------------------------------------------

mz: 85.08 count: 288(6.6) Range available: 4331 

        mz        perc
1   129.10  100.000000
2    86.09   98.958333
3   110.07   96.527778
4    84.08   95.138889
5   120.08   84.722222
6    84.04   84.027778
7   102.05   7

mz: 105.05 count: 108(1.1) Range available: 9423 

        mz       perc
1   129.10  97.222222
2   110.07  93.518519
3   120.08  92.592593
4   104.05  90.740741
5   101.07  76.851852
6   102.05  64.814815
7    86.09  62.037037
8    84.08  53.703704
9   112.08  51.851852
10   84.04  46.296296
11  115.08  45.370370
12   99.79  44.444444
------------------------------------------

mz: 105.06 count: 487(5.1) Range available: 9463 

        mz       perc
1   129.10  96.509240
2   110.07  92.607803
3   120.08  88.295688
4   101.07  76.591376
5   102.05  56.057495
6   129.06  49.075975
7   104.05  48.254620
8   112.65  43.737166
9    86.09  43.737166
10   99.79  41.067762
11  115.08  38.398357
12   84.08  36.755647
------------------------------------------

mz: 107.04 count: 153(1.6) Range available: 9478 

        mz       perc
1   129.10  94.771242
2   110.07  92.810458
3   120.08  90.196078
4   101.07  64.705882
5   102.05  59.477124
6   119.04  58.823529
7   129.06  50.326797
8   112.65 

mz: 125.07 count: 372(2.6) Range available: 13887 

        mz       perc
1   129.10  95.430108
2   120.08  81.989247
3   110.07  67.204301
4   101.07  44.892473
5   129.06  43.548387
6   104.05  38.709677
7   102.05  38.709677
8   115.08  38.440860
9   116.07  36.021505
10  112.65  35.483871
11  121.08  32.526882
12  127.08  30.376344
------------------------------------------

mz: 125.1 count: 248(1.7) Range available: 13898 

        mz       perc
1   129.10  94.758065
2   120.08  85.080645
3   110.07  74.596774
4   101.07  56.451613
5   102.05  51.209677
6    86.09  44.354839
7   104.05  42.741935
8   112.65  41.129032
9    84.08  40.322581
10   87.10  37.096774
11  129.06  36.693548
12   84.04  35.887097
------------------------------------------

mz: 126.05 count: 878(6.2) Range available: 13956 

        mz       perc
1   129.10  95.102506
2   120.08  83.712984
3   110.07  70.045558
4   101.07  48.974943
5   102.05  46.241458
6   129.06  44.191344
7   112.65  37.813212
8   115.0

In [10]:
selected_res=[]
for k in dict_res.keys():
    
    temp=(pd.DataFrame(np.unique(dict_res[k]["aas"] ,return_counts=True)).transpose())
    temp.columns=["aa","perc"]
    temp.perc=100*temp.perc/dict_res[k]["count"]
    temp=temp.sort_values(by="perc",ascending=False)
    temp=(temp[temp.perc>30].reset_index(drop=True))
    if (temp.shape[0]>0):
        print("mz:",k,"count:",f'{dict_res[k]["count"]}({int(1000*dict_res[k]["count"]/dict_res[k]["total"])/10})',"Range available:",dict_res[k]["total"],"\n")
        print(temp.head(9))
        print("------------------------------------------\n")
  

mz: 65.39 count: 185(100.0) Range available: 185 

  aa       perc
0  L   69.72973
1  A  58.918919
2  V  51.351351
3  K  50.810811
4  G  46.486486
5  R  45.405405
6  F  42.702703
7  I  36.216216
8  P  31.351351
------------------------------------------

mz: 69.07 count: 383(76.1) Range available: 503 

  aa       perc
0  I  71.801567
1  L  65.535248
2  K   58.48564
3  A  50.913838
4  V  48.563969
5  G  46.997389
6  R  38.120104
7  F  35.770235
8  S  32.114883
------------------------------------------

mz: 70.02 count: 135(23.3) Range available: 577 

  aa       perc
0  L  68.148148
1  K  54.074074
2  A  48.888889
3  V  45.185185
4  S  45.185185
5  R  44.444444
6  N  41.481481
7  F       40.0
8  G       40.0
------------------------------------------

mz: 70.06 count: 1265(98.9) Range available: 1279 

  aa       perc
0  L  66.798419
1  A  53.043478
2  V  52.490119
3  K  50.355731
4  R  48.458498
5  G    45.6917
6  I  42.766798
7  F  35.968379
8  S  34.071146
-------------------------

mz: 89.07 count: 204(3.6) Range available: 5553 

  aa       perc
0  A  74.509804
1  Q  62.745098
2  L   61.27451
3  R  51.960784
4  K  50.980392
5  G  49.509804
6  V  47.058824
7  I  42.647059
8  E  41.176471
------------------------------------------

mz: 91.05 count: 1242(21.6) Range available: 5747 

  aa       perc
0  L  64.814815
1  K  54.347826
2  Y  53.140097
3  G  52.093398
4  A  49.838969
5  V  48.309179
6  D  45.652174
7  R  45.652174
8  E  45.491143
------------------------------------------

mz: 93.04 count: 463(7.9) Range available: 5851 

  aa       perc
0  H  79.049676
1  L  63.066955
2  R  53.347732
3  G  52.267819
4  V  50.539957
5  A  50.323974
6  D  48.164147
7  E  47.516199
8  K  46.436285
------------------------------------------

mz: 94.17 count: 103(1.7) Range available: 5874 

  aa       perc
0  L  63.106796
1  G  58.252427
2  V  54.368932
3  A  53.398058
4  R  51.456311
5  S  50.485437
6  K  47.572816
7  E  46.601942
8  F  42.718447
--------------------------

mz: 128.17 count: 3566(24.3) Range available: 14657 

  aa       perc
0  L  74.172743
1  A  59.197981
2  V   59.08581
3  G  58.917555
4  E  57.571509
5  S  54.234436
6  K  53.673584
7  D  51.037577
8  I  51.009534
------------------------------------------

mz: 128.18 count: 3357(22.4) Range available: 14925 

  aa       perc
0  L  71.075365
1  V  60.857909
2  A   60.82812
3  E  58.623771
4  G  57.700328
5  K  54.006553
6  S  52.755436
7  I  50.819184
8  D  50.551087
------------------------------------------

mz: 128.19 count: 652(4.3) Range available: 14978 

  aa       perc
0  L  68.404908
1  A  63.343558
2  G  59.969325
3  V  58.588957
4  S  54.754601
5  E  54.601227
6  K   53.98773
7  T  49.233129
8  D  48.312883
------------------------------------------

mz: 129.06 count: 5270(34.7) Range available: 15177 

  aa       perc
0  L  71.461101
1  A  62.087287
2  E  59.848197
3  V  59.127135
4  G  57.874763
5  Q  55.616698
6  S   53.73814
7  K  53.017078
8  I  50.588235
--------------

================================
================

In [21]:
dict_res={x:{"count":0,"total":0,"aas":[],"peps":[]} for x in all_mz.mz}
count=0
for i in range(data.shape[0]): 
    temp=np.unique([ int(float(x)*100)/100 for x in data["values"][i].split("|")])
    temp=[x for x in temp if x in list(all_mz.mz)]
    if len(temp)==0:
        continue
        
    for mz in all_mz[all_mz.mz>= min(temp)].mz:
        dict_res[mz]["total"]+=1        
    
    for mz in temp:
        dict_res[mz]["count"]+=1
        dict_res[mz]["aas"]+=list(np.unique(list(data["peptide"][i])))
        dict_res[mz]["peps"]+=[data["peptide"][i]]
    

In [22]:
res=[]
for char in aas:
    temp_count=0
    for i in range(data.shape[0]):
        if char in list(data.peptide[i]):
            temp_count+=1
    res.append(temp_count)
df=pd.DataFrame()
df["AA"]=aas
df["freq"]=res
df=df.sort_values(by="freq").reset_index(drop=True).transpose()
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
AA,W,C,M,H,Y,N,Q,F,P,R,T,D,I,S,K,E,G,A,V,L
freq,2091,2236,3742,4222,4934,5896,6217,6382,6755,7341,7548,7744,7757,8183,8201,8757,8850,9173,9283,11116


In [23]:
aa_dict_res={x:{"count":0, "mzs":{mzs:{"count":0,"total":0} for mzs in all_mz.mz} }  for x in aas}
count=0
for i in range(data.shape[0]): 
    temp=np.unique([ int(float(x)*100)/100 for x in data["values"][i].split("|")])
    temp=[x for x in temp if x in list(all_mz.mz)]
     
    if len(temp)==0:
        continue
        
    for c in np.unique(list(data.peptide[i])):
        
        aa_dict_res[c]["count"]+=1
        
        
        
        for mz in all_mz[all_mz.mz>= min(temp)].mz:
            if mz <= aas_residualmz[c]:
                aa_dict_res[c]['mzs'][mz]["total"]+=1        
    
        for mz in temp:
            if mz <= aas_residualmz[c]:
                aa_dict_res[c]['mzs'][mz]["count"]+=1
            
        
    if(i%1000==0):
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000


In [30]:
chars="K"
res=[]
for mz in all_mz.mz:
#     print(aa_dict_res["H"])
    aa_mz_percentage=0
    if aa_dict_res[chars]["mzs"][mz]['total'] >0:
        aa_mz_percentage=100*aa_dict_res[chars]["mzs"][mz]['count']/aa_dict_res[chars]["mzs"][mz]['total']
        
    singal_occurrence = 0
    if aa_dict_res[chars]['count'] >0:
        singal_occurrence=100*aa_dict_res[chars]["mzs"][mz]['count']/aa_dict_res[chars]['count']
    
    
        
    res.append([mz,aa_dict_res[chars]['count'],
                aa_dict_res[chars]["mzs"][mz]['count'],aa_dict_res[chars]["mzs"][mz]['total'],
                aa_mz_percentage   ,singal_occurrence,dict_res_mzs[mz]['count'], 
                100* aa_dict_res[chars]["mzs"][mz]['count']/dict_res_mzs[mz]['count'] ])
#     print(mz,'\t',100*aa_dict_res["H"]["mzs"][mz]['count']/aa_dict_res["H"]["mzs"][mz]['total'])
    
res=pd.DataFrame(res)
res.columns=["mz","total_aa_occur","actual_count","expected_count","aa_mz_percentage","singal_occurrence",
            "mz_freq","signal_strength"]
res=res.sort_values(by=["aa_mz_percentage"],ascending=False)
res.head(50)

,mz,total_aa_occur,actual_count,expected_count,aa_mz_percentage,singal_occurrence,mz_freq,signal_strength
0,65.39,8200,94,94,100.000000,1.146341,185,50.810811
35,84.08,8200,2253,2263,99.558109,27.475610,4008,56.212575
40,86.09,8200,2528,2569,98.404048,30.829268,4853,52.091490
3,70.06,8200,637,651,97.849462,7.768293,1265,50.355731
5,72.08,8200,781,816,95.710784,9.524390,1523,51.280368
73,110.07,8200,5172,5868,88.139059,63.073171,9840,52.560976
94,120.08,8200,6161,7276,84.675646,75.134146,11793,52.242856
33,84.04,8200,1774,2131,83.247302,21.634146,3459,51.286499
1,69.07,8200,224,274,81.751825,2.731707,383,58.485640
63,101.07,8200,3319,4707,70.512003,40.475610,6329,52.441144


In [33]:
all_data=pd.DataFrame()

for chars in aas: 
    res=[]
    for mz in all_mz.mz:
    #     print(aa_dict_res["H"])
        aa_mz_percentage=0
        if aa_dict_res[chars]["mzs"][mz]['total'] >0:
            aa_mz_percentage=100*aa_dict_res[chars]["mzs"][mz]['count']/aa_dict_res[chars]["mzs"][mz]['total']

        singal_occurrence = 0
        if aa_dict_res[chars]['count'] >0:
            singal_occurrence=100*aa_dict_res[chars]["mzs"][mz]['count']/aa_dict_res[chars]['count']



        res.append([mz,aa_dict_res[chars]['count'],
                    aa_dict_res[chars]["mzs"][mz]['count'],aa_dict_res[chars]["mzs"][mz]['total'],
                    aa_mz_percentage   ,singal_occurrence,dict_res_mzs[mz]['count'], 
                    100* aa_dict_res[chars]["mzs"][mz]['count']/dict_res_mzs[mz]['count'] ])
    #     print(mz,'\t',100*aa_dict_res["H"]["mzs"][mz]['count']/aa_dict_res["H"]["mzs"][mz]['total'])

    res=pd.DataFrame(res)
    res.columns=["mz","total_aa_occur","actual_count","expected_count","aa_mz_percentage","singal_occurrence",
                "mz_freq","signal_strength"]
    res=res.sort_values(by=["aa_mz_percentage"],ascending=False)
    res['AA']=[chars]*res.shape[0]
    all_data=pd.concat([all_data,res])

In [34]:
all_data

,mz,total_aa_occur,actual_count,expected_count,aa_mz_percentage,singal_occurrence,mz_freq,signal_strength,AA
0,65.39,9162,109,109,100.000000,1.189697,185,58.918919,A
3,70.06,9162,671,677,99.113737,7.323728,1265,53.043478,A
1,69.07,9162,195,269,72.490706,2.128356,383,50.913838,A
2,70.02,9162,66,304,21.710526,0.720367,135,48.888889,A
73,110.07,9162,0,0,0.000000,0.000000,9840,0.000000,A
...,...,...,...,...,...,...,...,...,...
67,104.04,4928,26,2581,1.007361,0.527597,122,21.311475,Y
69,105.05,4928,26,2659,0.977811,0.527597,108,24.074074,Y
97,124.03,4928,39,4242,0.919378,0.791396,118,33.050847,Y
105,128.08,4928,37,4420,0.837104,0.750812,128,28.906250,Y


In [53]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
le = LabelEncoder()
label=le.fit_transform(all_data.AA)
label
enc = OneHotEncoder(handle_unknown='ignore')
o_enc=enc.fit_transform(np.array(all_data.AA).reshape(-1,1))
o_enc=o_enc.toarray()
o_enc

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [56]:
all_data[all_data.mz==124.11]

,mz,total_aa_occur,actual_count,expected_count,aa_mz_percentage,singal_occurrence,mz_freq,signal_strength,AA
98,124.11,9162,0,0,0.000000,0.000000,158,0.000000,A
98,124.11,2230,0,0,0.000000,0.000000,158,0.000000,C
98,124.11,7735,0,0,0.000000,0.000000,158,0.000000,D
98,124.11,8747,75,7729,0.970371,0.857437,158,47.468354,E
98,124.11,6376,60,5709,1.050972,0.941029,158,37.974684,F
98,124.11,8838,0,0,0.000000,0.000000,158,0.000000,G
98,124.11,4214,46,3726,1.234568,1.091599,158,29.113924,H
98,124.11,7747,0,0,0.000000,0.000000,158,0.000000,I
98,124.11,8200,89,7336,1.213195,1.085366,158,56.329114,K
98,124.11,11103,0,0,0.000000,0.000000,158,0.000000,L


In [85]:
selected_res=[]
for k in dict_res.keys():
    
    possible_aa=list(set([key for key,val in aas_residualmz.items() if val >=k ])- set(['K','L','R']))
    
    temp=[]
    for i in range(len(possible_aa)):
        for j in range(i+1,len(possible_aa)):
            for pep in dict_res[k]["peps"]:
                if possible_aa[i] in pep and possible_aa[j] in pep:
                    temp.append("".join(sorted(possible_aa[i]+possible_aa[j])))               
    
    temp=(pd.DataFrame(np.unique(temp,return_counts=True)).transpose())
    temp.columns=["aa","perc"]
    temp.perc=100*temp.perc/dict_res[k]["count"]
    temp=temp.sort_values(by="perc",ascending=False)
    temp=(temp[temp.perc>50].reset_index(drop=True))
    if temp.shape[0]>0:
        print(k,dict_res[k]["count"],dict_res[k]["total"],"\t===>\t",100*dict_res[k]["count"]/dict_res[k]["total"],"\n")
        print(temp)
        print("------------------------------------------\n")
    
    
#     selected_res.append([k,dict_res[k]["count"],dict_res[k]["total"],100*dict_res[k]["count"]/dict_res[k]["total"]])

104.04 122 9187 	===>	 1.3279634265810385 

   aa       perc
0  EM  53.278689
------------------------------------------



In [16]:
[key for key,val in aas_residualmz.items() if val >=75 ]

['S',
 'P',
 'V',
 'T',
 'C',
 'I',
 'L',
 'N',
 'D',
 'Q',
 'K',
 'E',
 'M',
 'H',
 'F',
 'R',
 'Y',
 'W']